In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)


In [3]:
df = pd.read_parquet('df_3.parquet')

In [4]:
df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'far_price', 'near_price',
       'bid_price', 'ask_price',
       ...
       'stock_vs_index_wap_ratio_mean_300_480',
       'stock_vs_index_wap_ratio_std_300_480', 'near_price_min_300_480',
       'near_price_max_300_480', 'near_price_mean_300_480',
       'near_price_std_300_480', 'spread_min_300_480', 'spread_max_300_480',
       'spread_mean_300_480', 'spread_std_300_480'],
      dtype='object', length=160)

# Data Pre-processing

In [5]:
df.shape

(653184, 160)

In [6]:
df.head(2)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,far_price,near_price,bid_price,ask_price,ask_size,wap,target,time_id,row_id,spread,mid_price,wap_diff,is_buy_pressure,auction_signal_strength,rolling_avg_imbalance,seconds_to_close,spread_change,wap_velocity,wap_velocity_60s,wap_lag_60s,spread_lag_60s,imbalance_lag_60s,synthetic_index_wap,wap_velocity_lag_60s,spread_change_lag_60s,stock_vs_index_wap_ratio,window_label,wap_diff_sd_max,wap_diff_sd_mean,wap_diff_sd_min,wap_diff_sd_std,auction_signal_strength_sd_max,auction_signal_strength_sd_mean,auction_signal_strength_sd_min,auction_signal_strength_sd_std,stock_vs_index_wap_ratio_sd_max,stock_vs_index_wap_ratio_sd_mean,stock_vs_index_wap_ratio_sd_min,stock_vs_index_wap_ratio_sd_std,near_price_sd_max,near_price_sd_mean,near_price_sd_min,near_price_sd_std,spread_sd_max,spread_sd_mean,spread_sd_min,spread_sd_std,wap_diff_sw_max,wap_diff_sw_mean,wap_diff_sw_min,wap_diff_sw_std,auction_signal_strength_sw_max,auction_signal_strength_sw_mean,auction_signal_strength_sw_min,auction_signal_strength_sw_std,stock_vs_index_wap_ratio_sw_max,stock_vs_index_wap_ratio_sw_mean,stock_vs_index_wap_ratio_sw_min,stock_vs_index_wap_ratio_sw_std,near_price_sw_max,near_price_sw_mean,near_price_sw_min,near_price_sw_std,spread_sw_max,spread_sw_mean,spread_sw_min,spread_sw_std,wap_diff_all_max,wap_diff_all_mean,wap_diff_all_min,wap_diff_all_std,auction_signal_strength_all_max,auction_signal_strength_all_mean,auction_signal_strength_all_min,auction_signal_strength_all_std,stock_vs_index_wap_ratio_all_max,stock_vs_index_wap_ratio_all_mean,stock_vs_index_wap_ratio_all_min,stock_vs_index_wap_ratio_all_std,near_price_all_max,near_price_all_mean,near_price_all_min,near_price_all_std,spread_all_max,spread_all_mean,spread_all_min,spread_all_std,initial_0s_wap_diff,lag_wap_diff_sd,delta_wap_diff_sd,initial_0s_auction_signal_strength,lag_auction_signal_strength_sd,delta_auction_signal_strength_sd,initial_0s_stock_vs_index_wap_ratio,lag_stock_vs_index_wap_ratio_sd,delta_stock_vs_index_wap_ratio_sd,initial_0s_near_price,lag_near_price_sd,delta_near_price_sd,initial_0s_spread,lag_spread_sd,delta_spread_sd,window_beg_wap_diff_sw,delta_within_windowwap_diff_sw,window_beg_auction_signal_strength_sw,delta_within_windowauction_signal_strength_sw,window_beg_stock_vs_index_wap_ratio_sw,delta_within_windowstock_vs_index_wap_ratio_sw,window_beg_near_price_sw,delta_within_windownear_price_sw,window_beg_spread_sw,delta_within_windowspread_sw,wap_diff_rollmean_10,auction_signal_strength_rollmean_10,stock_vs_index_wap_ratio_rollmean_10,near_price_rollmean_10,spread_rollmean_10,window_label_order,wap_diff_min_0_300,wap_diff_max_0_300,wap_diff_mean_0_300,wap_diff_std_0_300,auction_signal_strength_min_0_300,auction_signal_strength_max_0_300,auction_signal_strength_mean_0_300,auction_signal_strength_std_0_300,stock_vs_index_wap_ratio_min_0_300,stock_vs_index_wap_ratio_max_0_300,stock_vs_index_wap_ratio_mean_0_300,stock_vs_index_wap_ratio_std_0_300,spread_min_0_300,spread_max_0_300,spread_mean_0_300,spread_std_0_300,wap_diff_min_300_480,wap_diff_max_300_480,wap_diff_mean_300_480,wap_diff_std_300_480,auction_signal_strength_min_300_480,auction_signal_strength_max_300_480,auction_signal_strength_mean_300_480,auction_signal_strength_std_300_480,stock_vs_index_wap_ratio_min_300_480,stock_vs_index_wap_ratio_max_300_480,stock_vs_index_wap_ratio_mean_300_480,stock_vs_index_wap_ratio_std_300_480,near_price_min_300_480,near_price_max_300_480,near_price_mean_300_480,near_price_std_300_480,spread_min_300_480,spread_max_300_480,spread_mean_300_480,spread_std_300_480
0,0,0,480,1268787.96,1,0.999491,1.000241,1.000241,0.999383,0.999491,18656.00,0.999472,-1.320243,48,0_480_0,0.000108,0.999437,0.000035,1,0.000019,432090.973333,60,0.000001,0.000111,0.000624,0.998848,0.000107,0.0,1.000209,-0.000178,1.110223e-16,0.999262,480_600,0.000081,-0.000006,-0.000119,0.000037,0.000254,0.000045,-0.000471,0.000125,1.000355,0.99

In [7]:
df.shape

(653184, 160)

In [8]:
null_counts = df.isna().sum()
cols_with_nulls = null_counts[null_counts > 1]
# Temporarily show all rows
with pd.option_context('display.max_rows', None):
    print(cols_with_nulls)

far_price    157
dtype: int64


# Model Pipeline

# Loading Packages

In [9]:
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error



## Data Pre-processing

In [10]:
print(df.columns.tolist())


['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'far_price', 'near_price', 'bid_price', 'ask_price', 'ask_size', 'wap', 'target', 'time_id', 'row_id', 'spread', 'mid_price', 'wap_diff', 'is_buy_pressure', 'auction_signal_strength', 'rolling_avg_imbalance', 'seconds_to_close', 'spread_change', 'wap_velocity', 'wap_velocity_60s', 'wap_lag_60s', 'spread_lag_60s', 'imbalance_lag_60s', 'synthetic_index_wap', 'wap_velocity_lag_60s', 'spread_change_lag_60s', 'stock_vs_index_wap_ratio', 'window_label', 'wap_diff_sd_max', 'wap_diff_sd_mean', 'wap_diff_sd_min', 'wap_diff_sd_std', 'auction_signal_strength_sd_max', 'auction_signal_strength_sd_mean', 'auction_signal_strength_sd_min', 'auction_signal_strength_sd_std', 'stock_vs_index_wap_ratio_sd_max', 'stock_vs_index_wap_ratio_sd_mean', 'stock_vs_index_wap_ratio_sd_min', 'stock_vs_index_wap_ratio_sd_std', 'near_price_sd_max', 'near_price_sd_mean', 'near_price_sd_min', 'near_price_sd_std',

In [11]:
df.head(1)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,far_price,near_price,bid_price,ask_price,ask_size,wap,target,time_id,row_id,spread,mid_price,wap_diff,is_buy_pressure,auction_signal_strength,rolling_avg_imbalance,seconds_to_close,spread_change,wap_velocity,wap_velocity_60s,wap_lag_60s,spread_lag_60s,imbalance_lag_60s,synthetic_index_wap,wap_velocity_lag_60s,spread_change_lag_60s,stock_vs_index_wap_ratio,window_label,wap_diff_sd_max,wap_diff_sd_mean,wap_diff_sd_min,wap_diff_sd_std,auction_signal_strength_sd_max,auction_signal_strength_sd_mean,auction_signal_strength_sd_min,auction_signal_strength_sd_std,stock_vs_index_wap_ratio_sd_max,stock_vs_index_wap_ratio_sd_mean,stock_vs_index_wap_ratio_sd_min,stock_vs_index_wap_ratio_sd_std,near_price_sd_max,near_price_sd_mean,near_price_sd_min,near_price_sd_std,spread_sd_max,spread_sd_mean,spread_sd_min,spread_sd_std,wap_diff_sw_max,wap_diff_sw_mean,wap_diff_sw_min,wap_diff_sw_std,auction_signal_strength_sw_max,auction_signal_strength_sw_mean,auction_signal_strength_sw_min,auction_signal_strength_sw_std,stock_vs_index_wap_ratio_sw_max,stock_vs_index_wap_ratio_sw_mean,stock_vs_index_wap_ratio_sw_min,stock_vs_index_wap_ratio_sw_std,near_price_sw_max,near_price_sw_mean,near_price_sw_min,near_price_sw_std,spread_sw_max,spread_sw_mean,spread_sw_min,spread_sw_std,wap_diff_all_max,wap_diff_all_mean,wap_diff_all_min,wap_diff_all_std,auction_signal_strength_all_max,auction_signal_strength_all_mean,auction_signal_strength_all_min,auction_signal_strength_all_std,stock_vs_index_wap_ratio_all_max,stock_vs_index_wap_ratio_all_mean,stock_vs_index_wap_ratio_all_min,stock_vs_index_wap_ratio_all_std,near_price_all_max,near_price_all_mean,near_price_all_min,near_price_all_std,spread_all_max,spread_all_mean,spread_all_min,spread_all_std,initial_0s_wap_diff,lag_wap_diff_sd,delta_wap_diff_sd,initial_0s_auction_signal_strength,lag_auction_signal_strength_sd,delta_auction_signal_strength_sd,initial_0s_stock_vs_index_wap_ratio,lag_stock_vs_index_wap_ratio_sd,delta_stock_vs_index_wap_ratio_sd,initial_0s_near_price,lag_near_price_sd,delta_near_price_sd,initial_0s_spread,lag_spread_sd,delta_spread_sd,window_beg_wap_diff_sw,delta_within_windowwap_diff_sw,window_beg_auction_signal_strength_sw,delta_within_windowauction_signal_strength_sw,window_beg_stock_vs_index_wap_ratio_sw,delta_within_windowstock_vs_index_wap_ratio_sw,window_beg_near_price_sw,delta_within_windownear_price_sw,window_beg_spread_sw,delta_within_windowspread_sw,wap_diff_rollmean_10,auction_signal_strength_rollmean_10,stock_vs_index_wap_ratio_rollmean_10,near_price_rollmean_10,spread_rollmean_10,window_label_order,wap_diff_min_0_300,wap_diff_max_0_300,wap_diff_mean_0_300,wap_diff_std_0_300,auction_signal_strength_min_0_300,auction_signal_strength_max_0_300,auction_signal_strength_mean_0_300,auction_signal_strength_std_0_300,stock_vs_index_wap_ratio_min_0_300,stock_vs_index_wap_ratio_max_0_300,stock_vs_index_wap_ratio_mean_0_300,stock_vs_index_wap_ratio_std_0_300,spread_min_0_300,spread_max_0_300,spread_mean_0_300,spread_std_0_300,wap_diff_min_300_480,wap_diff_max_300_480,wap_diff_mean_300_480,wap_diff_std_300_480,auction_signal_strength_min_300_480,auction_signal_strength_max_300_480,auction_signal_strength_mean_300_480,auction_signal_strength_std_300_480,stock_vs_index_wap_ratio_min_300_480,stock_vs_index_wap_ratio_max_300_480,stock_vs_index_wap_ratio_mean_300_480,stock_vs_index_wap_ratio_std_300_480,near_price_min_300_480,near_price_max_300_480,near_price_mean_300_480,near_price_std_300_480,spread_min_300_480,spread_max_300_480,spread_mean_300_480,spread_std_300_480
0,0,0,480,1268787.96,1,0.999491,1.000241,1.000241,0.999383,0.999491,18656.0,0.999472,-1.320243,48,0_480_0,0.000108,0.999437,0.000035,1,0.000019,432090.973333,60,0.000001,0.000111,0.000624,0.998848,0.000107,0.0,1.000209,-0.000178,1.110223e-16,0.999262,480_600,0.000081,-0.000006,-0.000119,0.000037,0.000254,0.000045,-0.000471,0.000125,1.000355,0.999

In [12]:
cols_to_drop=['time_id','row_id','window_label_order','window_label']
df = df.drop(columns=cols_to_drop)
df.shape

(653184, 156)

In [13]:
cat_cols = ['stock_id']
numeric_cols = [col for col in df.columns if col not in cat_cols + ['target']]

# Ensure categorical column is of type string or category
df['stock_id'] = df['stock_id'].astype(str)  # or .astype('category')

# Optionally check float precision — no forced casting needed
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col])  # keeps decimal precision

df['stock_id'] = df['stock_id'].astype(str)

In [14]:
all_features = numeric_cols + cat_cols

In [15]:
df.dtypes

stock_id                    object
date_id                      int64
seconds_in_bucket            int64
imbalance_size             float64
imbalance_buy_sell_flag      int64
                            ...   
near_price_std_300_480     float64
spread_min_300_480         float64
spread_max_300_480         float64
spread_mean_300_480        float64
spread_std_300_480         float64
Length: 156, dtype: object

## Train Test Split

In [16]:
# Step 1: Split data by days (chronologically)
unique_days = sorted(df['date_id'].unique())
split_index = int(len(unique_days) * 0.8)

In [17]:
# Step 0: Filter train set by date
train_days = sorted(df['date_id'].unique())[:split_index]
train_df = df[df['date_id'].isin(train_days)].copy()

# Step 1: Reset index so indexing is clean
train_df = train_df.reset_index(drop=True)

# Step 2: Separate X and y after resetting index
X_train = train_df.drop(columns=['target'])  # or ['target', 'date_id'] if date_id not needed as feature
y_train = train_df['target']

# Step 3: Store date_id separately for CV split
train_date_ids = train_df['date_id'].copy()  # cleanly indexed now


In [18]:
# Step 4: Determine test set days (remaining 20% of date_ids)
test_days = sorted(df['date_id'].unique())[split_index:]

# Step 5: Filter test set by date
test_df = df[df['date_id'].isin(test_days)].copy()
test_df = test_df.reset_index(drop=True)

# Step 6: Separate X and y for test set
X_test = test_df.drop(columns=['target'])  # or drop 'date_id' as well if not needed
y_test = test_df['target']


# Model Training 

**Define the Custom Time-Based CV Splitter**

In [19]:
def time_based_cv_from_date_ids(date_ids, n_splits=4):
    unique_dates = sorted(date_ids.unique())
    split_size = len(unique_dates) // (n_splits + 1)

    for i in range(1, n_splits + 1):
        train_days = unique_dates[: i * split_size]
        test_days  = unique_dates[i * split_size : (i + 1) * split_size]

        train_idx = date_ids[date_ids.isin(train_days)].index
        test_idx  = date_ids[date_ids.isin(test_days)].index
        yield train_idx, test_idx


In [20]:
time_cv = time_based_cv_from_date_ids(train_date_ids, n_splits=4)


**RandomizedSearchCV on Training Data
**

In [21]:
param_dist = {
    'learning_rate': np.linspace(0.01, 0.2, 20),
    'depth': np.arange(4, 10),
    'l2_leaf_reg': np.logspace(-1, 2, 10),
    'iterations': [100, 200, 300]
}
random_cv = RandomizedSearchCV(
    estimator=CatBoostRegressor(verbose=0, random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_absolute_error',
    cv=time_cv,
    verbose=2,
    n_jobs=-1
)

random_cv.fit(X_train, y_train, cat_features=cat_cols)

print("✅ Best Parameters from Random Search:")
print(random_cv.best_params_)

best_model = random_cv.best_estimator_

Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] END depth=4, iterations=200, l2_leaf_reg=2.1544346900318834, learning_rate=0.04; total time=  46.3s
[CV] END depth=4, iterations=300, l2_leaf_reg=4.6415888336127775, learning_rate=0.09; total time= 2.4min
[CV] END depth=4, iterations=100, l2_leaf_reg=1.0, learning_rate=0.09999999999999999; total time=  45.7s
[CV] END depth=4, iterations=300, l2_leaf_reg=4.6415888336127775, learning_rate=0.09; total time= 2.9min
[CV] END depth=4, iterations=100, l2_leaf_reg=1.0, learning_rate=0.09999999999999999; total time=  35.3s
[CV] END depth=4, iterations=300, l2_leaf_reg=4.6415888336127775, learning_rate=0.09; total time= 1.8min
[CV] END depth=5, iterations=100, l2_leaf_reg=0.1, learning_rate=0.19; total time=  40.2s
[CV] END depth=5, iterations=100, l2_leaf_reg=10.0, learning_rate=0.13; total time= 1.1min
[CV] END depth=6, iterations=100, l2_leaf_reg=10.0, learning_rate=0.17; total time= 1.3min
[CV] END depth=6, iterations=300, l2_

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


### Model Validation on Holdout

In [22]:
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f"✅ MAE on Test Set: {mae:.4f}")

✅ MAE on Test Set: 4.9806


In [23]:
import pickle

# Assuming your best model is in variable: best_model
with open('catboost_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [24]:
import pandas as pd

# Convert CV results to DataFrame
cv_results_df = pd.DataFrame(random_cv.cv_results_)

# Select relevant columns
columns = [
    'params', 
    'mean_test_score', 
    'std_test_score'
] + [col for col in cv_results_df.columns if col.startswith('split') and col.endswith('_test_score')]

# Sort by best mean_test_score (highest negative MAE = lowest actual MAE)
cv_results_df = cv_results_df[columns].sort_values(by='mean_test_score', ascending=False)

# Convert scores back to positive MAE for readability
cv_results_df['mean_MAE'] = -cv_results_df['mean_test_score']
cv_results_df['std_MAE'] = cv_results_df['std_test_score']
for col in columns:
    if col.startswith('split'):
        cv_results_df[col] = -cv_results_df[col]

# Show top 10 results
cv_results_df[['params', 'mean_MAE', 'std_MAE'] + [col for col in columns if col.startswith('split')]].head(10)


,params,mean_MAE,std_MAE,split0_test_score,split1_test_score,split2_test_score,split3_test_score
7,"{'learning_rate': 0.06999999999999999, 'l2_lea...",5.936981,0.453767,6.559909,6.181169,5.505357,5.501491
10,"{'learning_rate': 0.01, 'l2_leaf_reg': 0.21544...",5.942300,0.450035,6.550887,6.198508,5.500638,5.519169
12,"{'learning_rate': 0.13, 'l2_leaf_reg': 21.5443...",5.945197,0.449224,6.560857,6.188549,5.515918,5.515464
6,"{'learning_rate': 0.13, 'l2_leaf_reg': 10.0, '...",5.953466,0.449278,6.563198,6.206066,5.519645,5.524954
0,"{'learning_rate': 0.04, 'l2_leaf_reg': 2.15443...",5.956938,0.455234,6.584735,6.197226,5.510689,5.535102
15,"{'learning_rate': 0.05, 'l2_leaf_reg': 0.21544...",5.962125,0.463172,6.597345,6.212213,5.528947,5.509997
1,"{'learning_rate': 0.09999999999999999, 'l2_lea...",5.966617,0.448713,6.577990,6.215012,5.524571,5.548894
8,"{'learning_rate': 0.17, 'l2_leaf_reg': 10.0, '...",5.972635,0.453513,6.578592,6.241606,5.549631,5.520710
17,"{'learning_rate': 0.09, 'l2_leaf_reg': 1.0, 'i...",5.975485,0.459654,6.609562,6.217814,5.549627,5.524936
19,"{'learning_rate': 0.13, 'l2_leaf_reg': 21.5443...",5.975746,0.461868,6.582761,6.264590,5.535144,5.520489


In [28]:
# Parameters for gridsearch
# Convert to DataFrame
cv_results_df = pd.DataFrame(random_cv.cv_results_)

# Get top 5 by best (lowest) MAE
top_params = cv_results_df.sort_values(by='mean_test_score', ascending=False).head(3)



Create a Custom CatBoost Wrapper

In [29]:
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split

class CatBoostWithEarlyStop(BaseEstimator, RegressorMixin):
    def __init__(self, **params):
        self.params = params
        self.model = None

    def fit(self, X, y):
        # Internally split into train/val for early stopping
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, shuffle=False  # keep time order
        )

        self.model = CatBoostRegressor(
            **self.params,
            verbose=0,
            early_stopping_rounds=50,
            random_seed=42
        )

        self.model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            cat_features=self.params.get("cat_features", [])
        )
        return self

    def predict(self, X):
        return self.model.predict(X)

    def get_params(self, deep=True):
        return self.params.copy()

    def set_params(self, **params):
        self.params.update(params)
        return self


In [31]:
# Example: Extract ranges around best values
top_learning_rates = sorted(set(p['learning_rate'] for p in top_params['params']))
top_depths = sorted(set(p['depth'] for p in top_params['params']))
top_l2_regs = sorted(set(p['l2_leaf_reg'] for p in top_params['params']))
top_iterations = sorted(set(p['iterations'] for p in top_params['params']))

# These come from your previous top_params['params']
grid_params = {
    'learning_rate': top_learning_rates,
    'depth': top_depths,
    'l2_leaf_reg': top_l2_regs,
    'iterations': top_iterations,
    'cat_features': [cat_cols]  # Important! Needed inside wrapper
}
def time_based_cv_from_date_ids(date_ids, n_splits=4):
    unique_dates = sorted(date_ids.unique())
    split_size = len(unique_dates) // (n_splits + 1)

    for i in range(1, n_splits + 1):
        train_days = unique_dates[: i * split_size]
        test_days  = unique_dates[i * split_size : (i + 1) * split_size]

        train_idx = date_ids[date_ids.isin(train_days)].index
        test_idx  = date_ids[date_ids.isin(test_days)].index
        yield train_idx, test_idx

time_cv = time_based_cv_from_date_ids(train_date_ids, n_splits=4)

def time_based_cv_from_date_ids(date_ids, n_splits=4):
    unique_dates = sorted(date_ids.unique())
    split_size = len(unique_dates) // (n_splits + 1)

    for i in range(1, n_splits + 1):
        train_days = unique_dates[: i * split_size]
        test_days  = unique_dates[i * split_size : (i + 1) * split_size]

        train_idx = date_ids[date_ids.isin(train_days)].index
        test_idx  = date_ids[date_ids.isin(test_days)].index
        yield train_idx, test_idx

time_cv = time_based_cv_from_date_ids(train_date_ids, n_splits=4)
from sklearn.model_selection import GridSearchCV

grid_cv = GridSearchCV(
    estimator=CatBoostWithEarlyStop(),
    param_grid=grid_params,
    scoring='neg_mean_absolute_error',
    cv=time_cv,
    verbose=2,
    n_jobs=1
)

grid_cv.fit(X_train, y_train)

# Best model and params
print("Best Parameters from Grid Search:")
print(grid_cv.best_params_)
best_model = grid_cv.best_estimator_

Fitting 4 folds for each of 27 candidates, totalling 108 fits
[CV] END cat_features=['stock_id'], depth=9, iterations=100, l2_leaf_reg=0.21544346900318834, learning_rate=0.01; total time=   9.4s
[CV] END cat_features=['stock_id'], depth=9, iterations=100, l2_leaf_reg=0.21544346900318834, learning_rate=0.01; total time=  11.9s
[CV] END cat_features=['stock_id'], depth=9, iterations=100, l2_leaf_reg=0.21544346900318834, learning_rate=0.01; total time=  14.6s
[CV] END cat_features=['stock_id'], depth=9, iterations=100, l2_leaf_reg=0.21544346900318834, learning_rate=0.01; total time=  16.6s
[CV] END cat_features=['stock_id'], depth=9, iterations=100, l2_leaf_reg=0.21544346900318834, learning_rate=0.06999999999999999; total time=   9.7s
[CV] END cat_features=['stock_id'], depth=9, iterations=100, l2_leaf_reg=0.21544346900318834, learning_rate=0.06999999999999999; total time=  12.1s
[CV] END cat_features=['stock_id'], depth=9, iterations=100, l2_leaf_reg=0.21544346900318834, learning_rate=0.

In [34]:
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

grid_cv.best_params_

{'cat_features': ['stock_id'],
 'depth': 9,
 'iterations': 200,
 'l2_leaf_reg': 46.41588833612777,
 'learning_rate': 0.06999999999999999}

## Retraining with Best Paramaters

In [33]:

X_full = pd.concat([X_train, X_test], axis=0)
y_full = pd.concat([y_train, y_test], axis=0)

# Step 2: Retrain best model on full data
final_model = CatBoostRegressor(
    **grid_cv.best_params_,
    verbose=0,
    random_state=42
)
final_model.fit(X_full, y_full, cat_features=cat_cols)

# Step 3: Save to pickle
with open("final_catboost_model.pkl", "wb") as f:
    pickle.dump(final_model, f)

# Light GBM Model


In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
# Identify categorical columns
cat_cols = ['stock_id']  # Add more if needed

# Convert object-type columns to category dtype
for col in cat_cols:
    X_train[col] = X_train[col].astype('category')

def time_based_cv_from_date_ids(date_ids, n_splits=3):
    unique_dates = sorted(date_ids.unique())
    split_size = len(unique_dates) // (n_splits + 1)

    for i in range(1, n_splits + 1):
        train_days = unique_dates[: i * split_size]
        test_days  = unique_dates[i * split_size : (i + 1) * split_size]

        train_idx = date_ids[date_ids.isin(train_days)].index
        test_idx  = date_ids[date_ids.isin(test_days)].index
        yield train_idx, test_idx


time_cv = time_based_cv_from_date_ids(train_date_ids, n_splits=4)

# Define the parameter distribution
param_dist = {
    'learning_rate': [0.01, 0.05],
    'reg_lambda': [0.1, 1, 5],

    'max_depth': np.arange(4, 10),
    'n_estimators': [100, 200, 300]
}

# Initialize the LightGBM regressor
lgb_model = lgb.LGBMRegressor(random_state=42,  
    cat_l2=10.0)

# Set up RandomizedSearchCV
random_cv = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_absolute_error',
    cv=time_cv,  # Ensure time_cv is defined appropriately
    verbose=2,
    n_jobs=1,
     
    return_train_score=True
)

# Fit the model
random_cv.fit(X_train, y_train,categorical_feature=cat_cols)

# Display the best parameters
print("✅ Best Parameters from Random Search:")
print(random_cv.best_params_)


Fitting 4 folds for each of 20 candidates, totalling 80 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048488 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 37782
[LightGBM] [Info] Number of data points in the train set: 100695, number of used features: 154
[LightGBM] [Info] Start training from score 0.032417
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[CV] END learning_rate=0.05, max_depth=6, n_estimators=200, reg_lambda=0.1; total time=   4.9s
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 38006
[LightGBM] [Info] Number of data points in the train set: 203280, number of used features: 154
[LightGBM] [Info] Start training from score -0.041349
[CV] END learn

In [51]:
best_model = random_cv.best_estimator_
for col in cat_cols:
    X_test[col] = X_test[col].astype('category')
    X_test[col] = X_test[col].cat.set_categories(X_train[col].cat.categories)

y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f" MAE on Test Set: {mae:.4f}")

 MAE on Test Set: 5.0012
